# Objective

Create a script that can be used for updating the weekly board report

## Input

Merged Hepstar and Travix data - HS_clean

# Change log

- 2020/05/11: NM - Created separate dataset with most used segments

# Preparation

## Import required Pythmon packages

In [5]:
import numpy as np
import pandas as pd

In [6]:
from datetime import date
from datetime import datetime, timedelta
import time
import pytz
import glob

In [7]:
#%load_ext autotime
#%load_ext pycodestyle_magic
#%flake8_on

## Import data

In [12]:
# Network locations for data import & export
input_TravixDatadump_BI = r'\\192.168.140.37\\Business-Share\\bi-report\\Data analysis resources\\Travix\\DatadumpTravix\\'
Hepstar_path =r'\\192.168.140.37\\Business-Share\\bi-report\\'
input_BI_drive = r'\\192.168.140.37\Business-Share\BI resources\Dashboards\Travix\# Data\\'
export_BI_drive = r'\\192.168.140.37\Business-Share\BI resources\Dashboards\Travix\# Data\\'
export_NL_BI_drive =  r'\\192.168.140.37\\Business-Share\\BI resources\\Dashboards\\Travix\\# Travix NL\\# Data\\'

In [9]:
 # import HS clean data
HS_df1 = pd.read_csv(input_BI_drive+'20190701-present HS_clean_df.csv', error_bad_lines=False,low_memory=False)
HS_df2 = pd.read_csv(input_BI_drive+'20180501-20190630 HS_clean_df.csv', error_bad_lines=False,low_memory=False)

In [10]:
# Merge Travix csv files in folders with other csv files not in folders
HS_df = pd.concat([HS_df1, HS_df2], sort = False)

In [23]:
# make a copy of the dataset
HS_T_df = HS_df.copy()
# make a copy of the data set
HS_T_df = HS_T_df.copy()

# Data cleaning

In [24]:
# Change date variable
HS_T_df['date'] = pd.to_datetime(HS_T_df['date'])

In [25]:
#Define the date we went live for NL channel 
Start_date = ('2018-06-01 00:00:00')

In [26]:
# Calculate the travel duration days so as to create travel duration bracket for additional metrics 
# change start and end date to datetime
HS_T_df['startdate'] = pd.to_datetime(HS_T_df['startdate'])
HS_T_df['enddate'] = pd.to_datetime(HS_T_df['enddate'])

# Create travel duration brac
# Calculate duration variable
HS_T_df['Travelduration_recalc'] = HS_T_df['enddate'] - HS_T_df['startdate']

#converting number of days to integer type
HS_T_df['Travelduration_recalc']=HS_T_df['Travelduration_recalc'].dt.days

#Check for missing data
HS_T_df[HS_T_df['Travelduration_recalc'].isnull()]['triptype'].value_counts()

oneway    675777
Name: triptype, dtype: int64

In [27]:
# Select only NL channels
#HS_T_NL_df=HS_T_df.loc[HS_T_df['Brand/Channel'].isin(['CheapticketsNL','BudgetAir','Vliegwinkel'])]
HS_T_NL_df=HS_T_df.loc[HS_T_df['channel'].isin(['CheapticketsNL','BudgetAir','Vliegwinkel'])]

In [28]:
#make a copy of the dataset
HS_T_NL_df=HS_T_NL_df.copy()

In [29]:
# Remove August 20-22 because the session ID in the Travix datadump doesn't match for those days. Session ID can be added again when data issue has been resolved
HS_T_NL_df.drop(HS_T_NL_df[HS_T_NL_df['date'] == '2018-08-20'].index, inplace=True)
HS_T_NL_df.drop(HS_T_NL_df[HS_T_NL_df['date'] == '2018-08-21'].index, inplace=True)
HS_T_NL_df.drop(HS_T_NL_df[HS_T_NL_df['date'] == '2018-08-22'].index, inplace=True)
# Remove Nov 1 and 2 due to Travix deployment failure
HS_T_NL_df.drop(HS_T_NL_df[HS_T_NL_df['date'] == '2018-11-01'].index, inplace=True)
HS_T_NL_df.drop(HS_T_NL_df[HS_T_NL_df['date'] == '2018-11-02'].index, inplace=True)

In [31]:
# Calculate the number of days to departure
HS_T_NL_df['startdate'] = pd.to_datetime(HS_T_NL_df['startdate'])
HS_T_NL_df['enddate'] = pd.to_datetime(HS_T_NL_df['enddate'])
HS_T_NL_df['daystodep']=HS_T_NL_df['startdate']-HS_T_NL_df['date']

In [32]:
# Check if startdate is before booking date
#HS_df[HS_df['date']>HS_df['startdate']]

In [33]:
# Drop if startdate is before booking date
HS_T_NL_df.drop(HS_T_NL_df[HS_T_NL_df['date']>HS_T_NL_df['startdate']].index, inplace=True)

In [34]:
#converting number of days to integer type
HS_T_NL_df['daystodep']=HS_T_NL_df['daystodep'].dt.days

In [35]:
#Select relevant columns
HS_T_BR_df=HS_T_NL_df[['date','channel','trafficcode','travelitemvalue', 'triptype','tagresidency','Main_products_count','premium_sum','tax_sum', 'totalcost_sum',
'TR_Haul','premium58%','destination_region','Destination subregion','Origin region','Origin subregion','haul_type','World_Europe','HS_purchase','distributor_purchase',
'product_type','masterpolicyfee_ex_tax&HS','HS_purchased_dist_revenue','Dist_vs_Hepstar','Order_week','Order_year','Order_month','Pricecalls','Travelduration_recalc','daystodep']]

In [36]:
#Create a copy of the data
HS_T_BR_df=HS_T_BR_df.copy()

In [37]:
#rename distributor to control
HS_T_BR_df['Dist_vs_Hepstar'].replace('Distributor', 'Control',inplace=True)

In [39]:
#Exclude Refund protect data
HS_T_BR_df=HS_T_BR_df[~HS_T_BR_df['trafficcode'].str.contains('RP')]

In [40]:
#Check if there is any  missing data
HS_T_BR_df.isnull().sum()

date                               0
channel                            0
trafficcode                        0
travelitemvalue                   46
triptype                           0
tagresidency                    2449
Main_products_count            44908
premium_sum                        0
tax_sum                            0
totalcost_sum                      0
TR_Haul                      1060094
premium58%                         0
destination_region                 0
Destination subregion              0
Origin region                      0
Origin subregion                   0
haul_type                          0
World_Europe                       0
HS_purchase                        0
distributor_purchase               0
product_type                       0
masterpolicyfee_ex_tax&HS          0
HS_purchased_dist_revenue          0
Dist_vs_Hepstar                    0
Order_week                         0
Order_year                         0
Order_month                        0
P

In [41]:
#Fill missing values with n/a
HS_T_BR_df.fillna(0, inplace=True)

In [42]:
#Check if there are still any missing values
HS_T_BR_df.isnull().sum()

date                         0
channel                      0
trafficcode                  0
travelitemvalue              0
triptype                     0
tagresidency                 0
Main_products_count          0
premium_sum                  0
tax_sum                      0
totalcost_sum                0
TR_Haul                      0
premium58%                   0
destination_region           0
Destination subregion        0
Origin region                0
Origin subregion             0
haul_type                    0
World_Europe                 0
HS_purchase                  0
distributor_purchase         0
product_type                 0
masterpolicyfee_ex_tax&HS    0
HS_purchased_dist_revenue    0
Dist_vs_Hepstar              0
Order_week                   0
Order_year                   0
Order_month                  0
Pricecalls                   0
Travelduration_recalc        0
daystodep                    0
dtype: int64

In [43]:
#rename distributor to control
HS_T_BR_df['Dist_vs_Hepstar'].replace('Distributor', 'Control',inplace=True)

### Create top10 Tagresidency column

In [44]:
# Create variable with top 10 residency countries
# Create a list of top 10 residency countries
Top10residency = HS_T_BR_df['tagresidency'].value_counts()[:10].index.tolist()
# Put 2 code  top 10 residency countries using list
HS_T_BR_df["Top10residency"] = np.where(HS_T_BR_df["tagresidency"].isin(Top10residency), HS_T_BR_df["tagresidency"], "SubTop10")

In [45]:
#Checks
HS_T_BR_df["Top10residency"].value_counts()

nl          1497119
be            33754
SubTop10      26263
de             3900
us             3411
es             2603
0              2449
gb             2194
fr             1658
ch             1344
ro             1213
Name: Top10residency, dtype: int64

## Create pivot table 

In [46]:
#Create a pivot table that groups data per year, week, haul and policy type
HS_T_BR_pv_df=HS_T_BR_df.pivot_table(index=['date','Order_year','Order_week','Order_month','TR_Haul', 'channel',
'triptype', 'Top10residency','Main_products_count','destination_region',
'Destination subregion','Origin region','Origin subregion','haul_type','World_Europe','product_type','Dist_vs_Hepstar'], aggfunc='sum')

In [47]:
#Reset the index for the pivot table 
HS_T_BR_pv_df.reset_index(inplace=True)

In [48]:
#Compare original data with the pivot data 
HS_T_BR_pv_df['premium_sum'].sum()-HS_T_BR_df['premium_sum'].sum()

4.656612873077393e-10

In [49]:
#HS_T_BR_pv_df[HS_T_BR_pv_df['date'].max() - pd.to_timedelta("28day")]

In [50]:
HS_T_BR_pv_df['date'].max()

Timestamp('2020-05-10 00:00:00')

In [119]:
#Select last 4 weeks data for invesigation purposes
HS_T_BR_4wks_df=HS_T_BR_pv_df[HS_T_BR_pv_df['Order_year']==2020]
HS_T_BR_4wks_df=HS_T_BR_4wks_df[HS_T_BR_4wks_df['Order_week'] >= HS_T_BR_4wks_df['Order_week'].max()-4]

In [120]:
#Export data
HS_T_BR_pv_df.to_csv(export_NL_BI_drive+'TravixNL Board update pivoted data.csv',index=True)
HS_T_BR_4wks_df.to_csv(export_NL_BI_drive+'TravixNL Board update data last 4 weeks.csv',index=True)
#HS_T_pivot_df.to_csv(export_BI_drive+'20190620 Board update grouped data.csv',index=True)

# Board Report update workbook

In [54]:
HS_T_BR_update_df=HS_T_BR_df[['date','Dist_vs_Hepstar','World_Europe', 'triptype', 'premium58%', 'Pricecalls']]

In [55]:
HS_T_BR_update_df=HS_T_BR_update_df.copy()

In [56]:
#Select relevant columns for the board report pivot tables
HS_T_BR_update_df= HS_T_BR_update_df[['date','Dist_vs_Hepstar','World_Europe', 'triptype', 'premium58%', 'Pricecalls']]

In [57]:
HS_T_BR_update_df=HS_T_BR_update_df.copy()

In [58]:
HS_T_BR_update_df['date'] = pd.to_datetime(HS_T_BR_update_df['date'])

In [59]:
#Create columns that indicate start and end date
HS_T_BR_update_df['End_date']=HS_T_BR_update_df['date'].max()
HS_T_BR_update_df['Start_date']=HS_T_BR_update_df['date'].min()

## Overall KPI performance Distr vs HS

### Overall

In [60]:
#Select relevant columns
Dist_vs_HS_df =HS_T_BR_update_df[['Dist_vs_Hepstar', 'premium58%', 'Pricecalls','Start_date', 'End_date']]

In [61]:
#Create a pivot table for the overall KPI perfomance with all the data
Dist_vs_HS_pivot_df = Dist_vs_HS_df.pivot_table(index=['Dist_vs_Hepstar','Start_date', 'End_date'], aggfunc='sum')

In [62]:
#Reset the index for the pivot table 
Dist_vs_HS_pivot_df.reset_index(inplace=True)

In [63]:
#Set index to distributor vs Hepstar column
Dist_vs_HS_pivot_df.set_index('Dist_vs_Hepstar', inplace=True)

In [64]:
#Create a column that indicates RPB per distributor
Dist_vs_HS_pivot_df['RPA']=Dist_vs_HS_pivot_df['premium58%']/Dist_vs_HS_pivot_df['Pricecalls']

### Last 30 days

In [65]:
#Select last 30 days of data
Dist_vs_HS_30days_df = HS_T_BR_update_df[HS_T_BR_update_df['date'] > HS_T_BR_update_df['date'].max() - pd.to_timedelta("30day")]

In [66]:
#Check the number of selected days
Dist_vs_HS_30days_df['date'].nunique()

30

In [67]:
Dist_vs_HS_30days_df['date'].max()

Timestamp('2020-05-10 00:00:00')

In [68]:
#Select relevant columns
Dist_vs_HS_30days_df =Dist_vs_HS_30days_df[['Dist_vs_Hepstar','premium58%', 'Pricecalls','Start_date', 'End_date']]

In [69]:
#Create a pivot table
Dist_vs_HS_pv_30days_df= Dist_vs_HS_30days_df.pivot_table(index=['Dist_vs_Hepstar','Start_date', 'End_date' ], aggfunc='sum')

In [70]:
#Reset the index for the pivot table 
Dist_vs_HS_pv_30days_df.reset_index(inplace=True)

In [71]:
#rename Hepstar to Hepstar last 30 days to distinguish between the two 
Dist_vs_HS_pv_30days_df['Dist_vs_Hepstar'].replace('Hepstar', 'Hepstar_last_30 days',inplace=True)

In [72]:
#Set index to distributor vs Hepstar column
Dist_vs_HS_pv_30days_df.set_index('Dist_vs_Hepstar', inplace=True)

In [73]:
#Create a column that indicates RPB per region
Dist_vs_HS_pv_30days_df['RPA']=Dist_vs_HS_pv_30days_df['premium58%']/Dist_vs_HS_pv_30days_df['Pricecalls']

### Merge overall and last 30 days data 

In [74]:
#Combine overall and last 30 days data  to one dataframe
Dist_vs_HS_all_rows_df = pd.concat([Dist_vs_HS_pivot_df, Dist_vs_HS_pv_30days_df],sort=False)

In [75]:
Dist_vs_HS_all_rows_df

,Start_date,End_date,Pricecalls,premium58%,RPA
Dist_vs_Hepstar,,,,,
Control,2018-05-03,2020-05-10,143816,9.816247e+04,0.682556
Hepstar,2018-05-03,2020-05-10,1432092,1.063358e+06,0.742521
Hepstar_last_30 days,2018-05-03,2020-05-10,2992,3.385796e+03,1.131616


In [77]:
# Export data
#Dist_vs_HS_all_rows_df.to_csv(export_local+'TravixNL_board_Control_inception and 30 days.csv',index=True)
Dist_vs_HS_all_rows_df.to_csv(export_NL_BI_drive+'TravixNL_board_Control_inception and 30 days.csv',index=True)

## KPI performance

### Overall

In [78]:
#Select relevant columns
KPI_perf_df= HS_T_BR_update_df[['World_Europe', 'premium58%', 'Pricecalls','Start_date', 'End_date']]

In [79]:
#Create a pivot table for the KPI perfomance with all the data
KPI_perf_pivot_df=KPI_perf_df.pivot_table(index=['World_Europe','Start_date', 'End_date'], aggfunc='sum')

In [80]:
#Reset the index for the pivot table 
KPI_perf_pivot_df.reset_index(inplace=True)

In [81]:
#Set index to World_Europe column
KPI_perf_pivot_df.set_index('World_Europe', inplace=True)

In [82]:
#Insert a row that sums values per columns
KPI_perf_pivot_df.loc['Overall']=KPI_perf_pivot_df.sum()

In [83]:
#Create a column that indicates RPB per region
KPI_perf_pivot_df['RPA']=KPI_perf_pivot_df['premium58%']/KPI_perf_pivot_df['Pricecalls']

In [84]:
KPI_perf_pivot_df

,Start_date,End_date,Pricecalls,premium58%,RPA
World_Europe,,,,,
Long haul,2018-05-03,2020-05-10,656080.0,7.554680e+05,1.151488
Short haul,2018-05-03,2020-05-10,919828.0,4.060523e+05,0.441444
Overall,NaT,NaT,1575908.0,1.161520e+06,0.737048


### Last 2 weeks

In [85]:
# Select data for the last 2 weeks 
KPI_perf_lst_2wks_df = HS_T_BR_update_df[HS_T_BR_update_df['date'] > HS_T_BR_update_df['date'].max() - pd.to_timedelta("14day")]

In [86]:
#Create a copy of the dataset
KPI_perf_lst_2wks_df=KPI_perf_lst_2wks_df.copy()

In [87]:
#Check the number of selected days
KPI_perf_lst_2wks_df['date'].nunique()

14

In [88]:
#Create a pivot table with last 2 weeks of data
KPI_perf_lst_2wks_pivot_df=KPI_perf_lst_2wks_df.pivot_table(index=['World_Europe','Start_date', 'End_date'], aggfunc='sum')

In [89]:
#Reset the index for the pivot table 
KPI_perf_lst_2wks_pivot_df.reset_index(inplace=True)

In [90]:
#Rename World Europe categories to match board report workbook
#rename World  to Long haul
KPI_perf_lst_2wks_pivot_df['World_Europe'].replace('Long haul', 'Long haul_2wks',inplace=True)
#rename Europe  to Short haul
KPI_perf_lst_2wks_pivot_df['World_Europe'].replace('Short haul', 'Short haul_2wks', inplace=True)

In [91]:
#Set index to World_Europe column
KPI_perf_lst_2wks_pivot_df.set_index('World_Europe', inplace=True)

In [92]:
#Insert a row that sums values per columns
KPI_perf_lst_2wks_pivot_df.loc['Overall_2wks']=KPI_perf_lst_2wks_pivot_df.sum()

In [93]:
#Create a column that indicates RPB per region
KPI_perf_lst_2wks_pivot_df['RPA']=KPI_perf_lst_2wks_pivot_df['premium58%']/KPI_perf_lst_2wks_pivot_df['Pricecalls']

### Merge overall and last 2 weeks KPI  data 

In [94]:
#Combine overall and last 30 days data  to one dataframe
KPI_perf_all_rows_df = pd.concat([KPI_perf_pivot_df, KPI_perf_lst_2wks_pivot_df],sort=False)

In [95]:
KPI_perf_all_rows_df

,Start_date,End_date,Pricecalls,premium58%,RPA
World_Europe,,,,,
Long haul,2018-05-03,2020-05-10,656080.0,7.554680e+05,1.151488
Short haul,2018-05-03,2020-05-10,919828.0,4.060523e+05,0.441444
Overall,NaT,NaT,1575908.0,1.161520e+06,0.737048
Long haul_2wks,2018-05-03,2020-05-10,646.0,8.299568e+02,1.284763
Short haul_2wks,2018-05-03,2020-05-10,1011.0,7.909170e+02,0.782312
Overall_2wks,NaT,NaT,1657.0,1.620874e+03,0.978198


### Last 3rd and 4th week

In [96]:
# Last 3rd week from current week
# Select data for the last 3rd and 4th week
KPI_perf_lst_3wks_df = HS_T_BR_update_df[HS_T_BR_update_df['date'] > HS_T_BR_update_df['date'].max() - pd.to_timedelta("28day")]
#Make a copy of the data
KPI_perf_lst_3wks_df=KPI_perf_lst_3wks_df.copy()

In [97]:
#Remove data for the last 2 weks so as to have data for the 3rd and 4th previous weeks from the current week
KPI_perf_lst_3wks_df=KPI_perf_lst_3wks_df[KPI_perf_lst_3wks_df['date'] <= KPI_perf_lst_3wks_df['date'].max() - pd.to_timedelta("14day")]

In [98]:
#Check the number of selected days
KPI_perf_lst_3wks_df['date'].nunique()

14

In [99]:
#Create a pivot table with last 2 weeks of data
KPI_perf_lst_3wks_pivot_df=KPI_perf_lst_3wks_df.pivot_table(index=['World_Europe','Start_date', 'End_date'], aggfunc='sum')

In [100]:
KPI_perf_lst_3wks_df['date'].min()

Timestamp('2020-04-13 00:00:00')

In [101]:
#Reset the index for the pivot table 
KPI_perf_lst_3wks_pivot_df.reset_index(inplace=True)

In [102]:
#Rename World Europe categories to match board report workbook
#rename World  to Long haul
KPI_perf_lst_3wks_pivot_df['World_Europe'].replace('Long haul', 'Long haul_last_3rd_wk',inplace=True)
#rename Europe  to Short haul
KPI_perf_lst_3wks_pivot_df['World_Europe'].replace('Short haul', 'Short haul_last_3rd_wk', inplace=True)

In [103]:
#Set index to World_Europe column
KPI_perf_lst_3wks_pivot_df.set_index('World_Europe', inplace=True)

In [104]:
#Insert a row that sums values per columns
KPI_perf_lst_3wks_pivot_df.loc['Overall_3rd_wk']=KPI_perf_lst_3wks_pivot_df.sum()

In [105]:
KPI_perf_lst_3wks_pivot_df

,Start_date,End_date,Pricecalls,premium58%
World_Europe,,,,
Long haul_last_3rd_wk,2018-05-03,2020-05-10,504.0,671.2108
Short haul_last_3rd_wk,2018-05-03,2020-05-10,716.0,1000.9814
Overall_3rd_wk,NaT,NaT,1220.0,1672.1922


In [106]:
#Create a column that indicates RPB per region
KPI_perf_lst_3wks_pivot_df['RPA']=KPI_perf_lst_3wks_pivot_df['premium58%']/KPI_perf_lst_3wks_pivot_df['Pricecalls']

In [107]:
KPI_perf_lst_3wks_pivot_df.to_csv(export_NL_BI_drive+'TravixNL_board_KPI_inception and last 3rd weeks.csv',index=True)

In [108]:
KPI_perf_lst_3wks_pivot_df

,Start_date,End_date,Pricecalls,premium58%,RPA
World_Europe,,,,,
Long haul_last_3rd_wk,2018-05-03,2020-05-10,504.0,671.2108,1.331767
Short haul_last_3rd_wk,2018-05-03,2020-05-10,716.0,1000.9814,1.398019
Overall_3rd_wk,NaT,NaT,1220.0,1672.1922,1.370649


In [109]:
#Combine overall and last 3 rd week data  to one dataframe
KPI_perf_all_rows_3rd_wk_df = pd.concat([KPI_perf_all_rows_df, KPI_perf_lst_3wks_pivot_df],sort=False)

In [110]:
# Export data
KPI_perf_all_rows_3rd_wk_df.to_csv(export_NL_BI_drive+'TravixNL_board_KPI_inception and last 2 weeks.csv',index=True)
#KPI_perf_all_rows_3rd_wk_df.to_csv(export_local+'TravixNL_board_KPI_inception and last 2 weeks.csv',index=True)